# Reliability of borrowers

## General information

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')

We study the general information about the available data.

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


The dataset is represented by 21,525 rows and 12 columns. From the sample above, it is already noticeable that there are implicit duplicates and anomalous values in it. There are also gaps in the `days_employed` and `total_income` columns.

## Data preprocessing

### Removing gaps

**Display the number of missing values for each column.**

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are missing values in two columns. One of them is `days_employed`. The gaps in this column will be dealt with in the next step. Another column with missing values, `total_income`, stores income data. The amount of income is most affected by the type of employment, so fill in the gaps in this column with the median value for each type from the `income_type` column. For example, for a person with an employment type of `employee`, the gap in the `total_income` column must be filled by the median income among all records of the same type.

In [6]:
for income_type in data.income_type.unique():
    #income_median = data[data.income_type == income_type].total_income.median()
    data.loc[(data.income_type == income_type) & (data.total_income.isna()), 'total_income'] = \
    data.loc[data.income_type == income_type, 'total_income'].median()
    #data.isna().sum()
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Handling anomalous values

Artifacts (anomalies) can occur in the data - values that do not reflect reality and appeared due to some kind of error. This artifact would be the negative number of days of work experience in the `days_employed` column. Let's process the values in this column: replace all negative values with positive ones using the `abs()` method.

In [7]:
data.days_employed = data.days_employed.abs()

**For each type of employment, we will display the median value of the work experience `days_employed` in days.**

In [8]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Two types (unemployed ['безработный'] and pensioners ['пенсионер']) will get abnormally large values. Correcting such values is difficult, so we will leave them as they are. Moreover, we will not need this column for research.

**Display the list of unique values of the `children` column.**

In [9]:
data.children.unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

There are two anomalous values in the `children` column. Let's remove the rows containing such anomalous values from the `data` dataframe.

In [10]:
data = data[~(data.children == -1)]
data = data[~(data.children == 20)]

### Removing gaps (continued)

Fill in the gaps in the `days_employed` column with the median values for each `income_type` employment type.

In [11]:
for income_type in data.income_type.unique():
    data.loc[((data.income_type == income_type) & (data.days_employed.isna())), 'days_employed'] = \
    data.loc[data.income_type == income_type, 'days_employed'].median()

Make sure all gaps are filled.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Changing data types

Let's replace the real data type in the `total_income` column with an integer one using the `astype()` method.

In [13]:
data.total_income = data.total_income.astype('int')

### Handling duplicates

Let's handle implicit duplicates in the `education` column. This column has the same values, but written differently: using uppercase and lowercase letters. Let's convert them to lower case.

In [14]:
data.education = data.education.str.lower()

**Display the number of duplicate rows in the data. If such lines are present, delete them.**

In [15]:
data.duplicated().sum()

71

In [16]:
data.drop_duplicates(inplace=True)

### Data categorization

Based on the ranges below, create a `total_income_category` column in the `data` dataframe with categories:

- 0–30000 - `'E'`;
- 30001–50000 - ``D'`;
- 50001–200000 - ``C'`;
- 200001–1000000 - ``B'`;
- 1000001 and above - `'A'`.

In [17]:
def categorize_income(income_value):
    if income_value <= 30000:
        return 'E'
    elif 30001 <= income_value <= 50000:
        return 'D'
    elif 50001 <= income_value <= 200000:
        return 'C'
    elif 200001 <= income_value <= 1000000:
        return 'B'
    elif income_value >= 1000001:
        return 'A'

In [18]:
data['total_income_category'] = data.total_income.apply(categorize_income)

**Display a list of unique loan purposes from the `purpose` column.**

In [19]:
data.purpose.unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Let's create a function that, based on the data from the `purpose` column, will form a new `purpose_category` column, which will include the following categories:**

- `'операции с автомобилем'` ['car operations'],
- `'операции с недвижимостью'` ['real estate transactions'],
- `'проведение свадьбы'` ['conducting a wedding'],
- `'получение образования'` ['getting an education'].

In [20]:
def categorize_purpose(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'жиль' in purpose or 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'

In [21]:
data['purpose_category'] = data.purpose.apply(categorize_purpose)

## Exploratory data analysis

### Is there a relationship between the number of children and loan repayment on time?

Let's see the borrowers with how many children are represented in the dataset.

In [22]:
data.children.unique()

array([1, 0, 3, 2, 4, 5])

All borrowers can be categorized by the number of children:
- **without children** (category `0`),
- **1-2 child(s)** (category `1`),
- **large family (3 or more children)** (category `2`).

Let's create the `categorize_children` function, which, based on the data from the `children` column, will form a new `children_category` column, which will include the categories described above.

In [23]:
def categorize_children(children):
    if children == 0:
        return 0
    elif 1 <= children <= 2:
        return 1
    else:
        return 2

Let's apply the function and check the result of the execution by displaying the first 10 rows of the dataset.

In [24]:
data['children_category'] = data.children.apply(categorize_children)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,0
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,2
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,0
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью,0
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью,0
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования,0
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы,1
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью,0


Let's see how the categories are presented.

In [25]:
data.children_category.value_counts()

0    14091
1     6860
2      380
Name: children_category, dtype: int64

We see that the number of borrowers without children is twice as many as those with one or two children. Parents with many children make up no more than 2% of the total number of borrowers.

Now let's group the borrowers into categories assigned based on the number of children, and calculate the relative number of delinquencies in payments in each of them.

In [26]:
data.groupby('children_category')['debt'].mean()

children_category
0    0.075438
1    0.093003
2    0.081579
Name: debt, dtype: float64

**Conclusion:** There is a noticeable peak in loan defaults for borrowers with `1-2 children` (almost 25% more than for people with `no children`).
"Borrowers with many children" are less likely to be late with payment payments, but still their share is 8% higher than that of "childless" ones.
Thus, it can be argued that the relationship between the number of children and the repayment of the loan on time **is present** and is significant.

### Is there a relationship between marital status and loan repayment on time?

Let's look at the distribution of the number of people in these categories.

In [27]:
data.family_status.value_counts()

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In the dataset, all categories are sufficiently represented, therefore, similarly to the previous calculation, we will group borrowers by marital status and consider the relative number of overdue payments by category.

In [28]:
data.groupby('family_status')['debt'].sum() / data.family_status.value_counts()

Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.093130
женат / замужем          0.075606
dtype: float64

**Conclusion:** The largest number of non-returns is observed among people in the category "Single" (9.8%). Common-law borrowers are almost as likely to default on time. On the contrary, widows and widowers most often pay off their loans on time (arrears occur in 6.6% of cases).

Thus, the relationship between marital status and repayment of the loan on time <b>is</b>. However, there is no direct connection with the fact that the lender has a partner, that is, a person who could potentially help with repaying the debt and repaying the loan on time.

For example, for the categories "Single / not married" and "civil marriage", as well as for "divorced" and "married / married", the shares of non-returns are quite close in value, although representatives of one category lead an independent life, and the other - a joint . At the same time, the difference between these pairs of categories is significant.

In other words, the circumstances of the creation or dissolution of an alliance between people rather than the very fact of its existence (divorce / death of a spouse, official marriage / cohabitation) more likely affect the likelihood of timely payment of a loan.

### Is there a relationship between income and loan repayment on time?

Category distribution:

In [29]:
data.total_income_category.value_counts()

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

We see that the categories of people with incomes up to 30 tons and from 1 million rubles are not sufficiently represented, so the conclusions for them may be erroneous.

Let's consider the relative number of delays for the previously selected salary categories (the `total_income_category` column of the dataframe).

In [30]:
data.groupby('total_income_category')['debt'].sum() / data.total_income_category.value_counts()

A    0.080000
B    0.070602
C    0.084982
D    0.060172
E    0.090909
dtype: float64

**Conclusion:**
Contrary to the intuitive assumption that wages will be inversely proportional to the number of loan arrears, people with a salary of `from 1 million rubles` per month are late with payments *a third more often* than those who earn `from 30 to 50 thousand rubles`. \
Borrowers who receive `up to 30 thousand rubles` per month, more than others have delinquencies on loans (*9.1%*).\
However, it should be noted once again that representatives of these categories make up only 0.1% of the total number of borrowers. Therefore, these statements require further study.

However, this hypothesis is also confirmed: the repayment of the loan on time really **depends** on the level of income.

### How do different purposes of a loan affect its repayment on time?

Category distribution:

In [31]:
data.purpose_category.value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

The categories are fairly broad. The conclusions drawn on the basis of such data can be considered with a high probability reflecting reality.

Let's calculate the relative number of delinquencies for borrowers grouped into categories by the purpose of the loan.

In [32]:
data.groupby('purpose_category')['debt'].sum() / data.purpose_category.value_counts()

операции с автомобилем      0.093480
операции с недвижимостью    0.072551
получение образования       0.092528
проведение свадьбы          0.079118
dtype: float64

**Conclusion:** The purpose of the loan **affects** the fact that it is repaid on time.

Payments on `auto- and educational loans` overstay equally often - in *9.3%* of cases.\
Borrowers who take money for `real estate transactions` and `weddings` repay the loan on time by *15 - 22%* more often. \
The fact that borrowers purchase physical objects (car, apartment) or services (education, wedding) with credit money does not affect the return of the amount on time.

### Possible causes of gaps in the source data

In the source data, gaps occur in the `days_employed` and `total_income` columns. Moreover, they are located in the same lines, and their numbers are equal.
- The most likely reason for their existence is the fact that the borrower does not have any work experience, and hence wages. To do this, let's check if there are rows in the dataset in which the value `0` occurs in the mentioned columns.

In [33]:
data[data.total_income == 0]['total_income'].sum()

0

In [34]:
data[data.days_employed == 0]['days_employed'].sum()

0.0

Neither the `days_employed` column nor the `total_income` column has values ​​equal to zero. That is, most likely, for borrowers who never worked, this data was simply not entered.
- There is no connection between the gaps and the values ​​of other columns - the lack of information about wages and length of service is found in all categories of citizens.
- Also, with a high degree of probability, it can be argued that there was a technical error in the formation of the table with the initial data (either some kind of failure during uploading the data for analysis, or when combining the data into a single file, which should also have been sent for analysis).

## General conclusion

**Based on the study, it can be argued that the loan repayment on time is affected by the number of children of the borrower, his income level, marital status, as well as the very purpose for which the money is taken.**

To understand the maximum spread of probabilities of not receiving money from the borrower on time, we calculate the indicators of untimely repayment of borrowed funds for the most reliable and unreliable categories of citizens from the point of view of the lender.

So, you can make a "portrait" of the most conscientious borrower:
- a person without children,
- widower / widow,
- earns from 30 to 50 thousand roubles,
- Takes out a loan for real estate transactions.

In [35]:
#Select rows that match all parameters
data_best = data[data.children_category == 0]
data_best = data_best[data_best.family_status_id == 2]
data_best = data_best[data_best.total_income_category == 'D']
data_best = data_best[data_best.purpose_category == 'операции с недвижимостью']

#Display the number of borrowers that fit the selected criteria
data_best.debt.count()

11

Of the entire data set, only 11 people fit the proposed parameters, which is 0.05% of the total number of borrowers, so no conclusions should be drawn based on this sample.\
Let's try to remove one of the parameters. We choose the one that cuts off the largest number of people.

In [36]:
data.groupby('children_category')['children_category'].count()

children_category
0    14091
1     6860
2      380
Name: children_category, dtype: int64

In [37]:
data.groupby('family_status_id')['family_status_id'].count()

family_status_id
0    12261
1     4134
2      951
3     1189
4     2796
Name: family_status_id, dtype: int64

In [38]:
data.groupby('total_income_category')['total_income_category'].count()

total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: total_income_category, dtype: int64

In [39]:
data.groupby('purpose_category')['purpose_category'].count()

purpose_category
операции с автомобилем       4279
операции с недвижимостью    10751
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

We see that of the parameters of interest to us, the income category `D` cuts off the most lines. Let's remove it and see how many people fall under the new filter.

In [40]:
#Select rows that match all parameters
data_best = data[data.children_category == 0]
data_best = data_best[data_best.family_status_id == 2]
data_best = data_best[data_best.purpose_category == 'операции с недвижимостью']

#Display the number of borrowers that fit the selected criteria
data_best.debt.count()

470

The sample consists of 470 people, i.e. 2.2% of the total. Let's look at the percentage of non-return on time among people who meet all these parameters.

In [41]:
data_best.debt.sum() / data_best.debt.count()

0.04680851063829787

For this sample, the percentage of non-repaid loans was only 4.7%.

Let's see what result the most "unreliable" category of citizens will have. Let's make their "portrait":
- 1-2 children,
- unmarried
- income up to 30 thousand roubles per month,
- purpose: car operations.

Based on previous experience, we will immediately consider which parameters cut off the sample as much as possible. It turns out that this is an income category `E`, that is, up to 30k rub.\
You can not get rid of this filter, but change the category to `C` (people with monthly income from 50k to 200k rub.), since the percentage of overdue loan payments for these two categories is close (9.1% versus 8.5%, respectively). Then the new set of parameters will be as follows:
- 1-2 children,
- unmarried / unmarried
- income from 50 to 200 thousand roubles per month,
- purpose: car operations.

In [42]:
#Select rows that match all parameters
data_worst = data[data.children_category == 1]
data_worst = data_worst[data_worst.family_status_id == 4]
data_worst = data_worst[data_worst.total_income_category == 'C']
data_worst = data_worst[data_worst.purpose_category == 'операции с автомобилем']

#Display the number of borrowers that fit the selected criteria
data_worst.debt.count()

89

The sample is still too small - only 0.4% of borrowers are represented in it. Nevertheless, we calculate the percentage of delays:

In [43]:
data_worst.debt.sum() / data_worst.debt.count()

0.1797752808988764

**Thus, in addition to the main conclusion that such parameters as the number of children, income level, marital status of the borrower and the purpose of the loan affect its repayment on time, some additional remarks can be made.**

Although when considering the "portraits" of citizens, the most reliable and unreliable from the point of view of the creditor, the samples were not representative enough to draw unambiguous conclusions, there is undoubtedly ground for further research.

**According to the few available data, it can be seen that a well-designed credit scoring model will allow avoiding significant risks when making positive decisions on issuing a loan.\
Thus, for a borrower that falls under the description of "unreliable", the probability of default on the loan is as much as 3.8 times greater than for one whom we have defined as "reliable".**